##Lecture 6 - Ngrams predictions

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
df = pd.read_csv("/content/drive/My Drive/NLP datasets/Consumer_Complaints_big.csv")
df.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,07/17/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,ALLY FINANCIAL INC.,TX,75035,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309495
1,07/17/2019,Credit card or prepaid card,General-purpose prepaid card,Trouble using the card,Problem using the card to withdraw money from ...,NaN,NaN,SQUARE INC,KS,674XX,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3310031
2,07/17/2019,Debt collection,Other debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,"Diversified Consultants, Inc.",FL,NaN,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309687
3,07/17/2019,Mortgage,VA mortgage,Trouble during payment process,NaN,NaN,NaN,"FLAGSTAR BANK, FSB",VA,22554,Servicemember,NaN,Web,07/17/2019,In progress,Yes,NaN,3308925
4,07/17/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes it acted appropriately as aut...,BYL Collection Services,TN,370XX,Servicemember,NaN,Web,07/17/2019,Closed with explanation,Yes,NaN,3308914


In [3]:
df1 = df[["Product", "Consumer complaint narrative"]]
df1["Product"] = df1["Product"].str.lower()
df1.isnull().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Product                              0
Consumer complaint narrative    938347
dtype: int64

In [4]:
df1 = df1.dropna()
df1.isnull().sum()
df1["Product"].value_counts()
df1 = df1.replace(to_replace =["credit card", "prepaid card"],  
                            value ="credit card or prepaid card")
df1 = df1.replace(to_replace ="credit reporting",
                            value ="credit reporting, credit repair services, or other personal consumer reports")
df1 = df1.replace(to_replace ="virtual currency",
                            value ="money transfer, virtual currency, or money service")
df1 = df1.replace(to_replace ="money transfers",
                            value ="money transfer, virtual currency, or money service")
df1 = df1.replace(to_replace =["student loan", "consumer loan", "vehicle loan or lease", "payday loan, title loan, or personal loan", "payday loan"],  
                            value ="loans")
df1 = df1.replace(to_replace ="checking or savings account",  
                            value ="bank account or service")
df1 = df1[df1.Product != 'money transfer, virtual currency, or money service']
df1 = df1[df1.Product != 'other financial service']
df1["Product"].value_counts()


credit reporting, credit repair services, or other personal consumer reports    129418
debt collection                                                                  89379
mortgage                                                                         54109
loans                                                                            44409
credit card or prepaid card                                                      43103
bank account or service                                                          28554
Name: Product, dtype: int64

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1["Product"] = le.fit_transform(df1["Product"])
df1.head()
df1["Product"].value_counts()
dfs = df1.sample(n = 1000, random_state = 42, replace=False)
dfs.head()
corpus = dfs["Consumer complaint narrative"].tolist()



In [7]:
##Text Preprocessing
import nltk
nltk.download()##all
from nltk.corpus import stopwords
from string import punctuation
stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
final_corpus= []
for i in range(len(corpus)):
    text = word_tokenize(corpus[i].lower())
    text = [t for t in text if len(t) > 2]
    text = [lemmatizer.lemmatize(y) for y in text if y not in stuff_to_be_removed]
    sent = " ".join(text)
    sent = " ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
    
    final_corpus.append(sent)
new_df = pd.DataFrame(final_corpus)
dfs = dfs.reset_index(drop=True)
new_df["Product"] = dfs["Product"]
new_df.columns = ["Consumer complaint narrative", "Product"]


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

In [8]:
new_df

,Consumer complaint narrative,Product
0,parent plus loan ' re taken loan 8 . 25 repaym...,4
1,loan sold within day / / issue / / date statem...,5
2,credit card closed account lost job come find ...,3
3,- dollar late fee equity line credit bought na...,0
4,filing complaint express policy raising intere...,1
...,...,...
995,actually method contact file dispute currently...,2
996,recently service connected veteran time good e...,2
997,fifth time update chase letter / / 2018 compla...,1
998,2015 bill pay 2000 . 00 bluebird account accou...,0


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(2,3))
vector= tfidf.fit_transform(new_df["Consumer complaint narrative"])
X = vector.toarray()
y = new_df["Product"]

from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.3,random_state=40)
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train,y_train)
y_pred=mnb.predict(X_test)
mnb.score(X_test, y_test)

0.38

In [10]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(y_test, y_pred)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))

precision: [0.   0.   0.38 0.   0.   0.  ]
recall: [0. 0. 1. 0. 0. 0.]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


##Web Scraping - Beautiful Soup

In [11]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup

In [0]:
news = "https://www.news18.com/news/world/un-chief-urges-india-pak-to-exercise-maximum-restraint-on-kashmir-invokes-shimla-agreement-2263887.html"

In [0]:
import urllib.request

In [0]:
request = urllib.request.Request(news)

In [0]:
response = urllib.request.urlopen(request)

In [0]:
content = response.read()

In [17]:
content

b'<!doctype html>\r\n<html><head><meta charset="utf-8"><link href=\'https://fonts.googleapis.com/css?family=Roboto:400,300italic,400italic,500,700\' rel=\'stylesheet\' type=\'text/css\'><title>UN Chief Urges India, Pak to Exercise \'Maximum Restraint\' on Kashmir, Invokes Simla Agreement - News18</title><meta name="description" content="The Secretary General Antonio Guterres did not offer his good offices nor did he make any offer to mediate between India and Pakistan on Kashmir." /><meta  name="keywords" content="Antonio Guterres, Article 370 revoked, J&amp;amp;K, jammu and kashmir, kashmir tensions, pakistan, shimla agreement, un, United nations" /><meta name="news_keywords" content="Antonio Guterres, Article 370 revoked, J&amp;amp;K, jammu and kashmir, kashmir tensions, pakistan, shimla agreement, un, United nations">\r\n<meta name="msapplication-starturl" content="https://www.news18.com/news/world/un-chief-urges-india-pak-to-exercise-maximum-restraint-on-kashmir-invokes-shimla-agre

In [0]:
soup = BeautifulSoup(content,'lxml')


In [19]:
data = soup.find_all("p")
for i in data:
  print(i.text)



Please check above checkbox.
News18.com
United Nations: UN chief Antonio Guterres on Thursday urged India and Pakistan to exercise "maximum restraint" and refrain from taking steps that could affect the status of Jammu and Kashmir, as he highlighted the Simla Agreement which rejects any third-party mediation on the issue.
The Secretary-General's remarks came after India on Monday revoked Article 370 to withdraw the special status to Jammu and Kashmir and bifurcated the state into two Union Territories - Jammu and Kashmir and Ladakh.
Pakistan termed the Indian action as "unilateral and illegal", and said it will take the matter to the UN Security Council.
"The Secretary-General has been following the situation in Jammu and Kashmir with concern and makes an appeal for maximum restraint," Guterres' spokesman Stephane Dujarric said here.
Dujarric specifically said that the Secretary-General "also recalls the 1972 Agreement on bilateral relations between India and Pakistan, also known as 

In [20]:
text = data[5:-1]
text = [text[i].text for i in range(len(text))]
finalnews = " ".join(text)
finalnews


'The Secretary-General\'s remarks came after India on Monday revoked Article 370 to withdraw the special status to Jammu and Kashmir and bifurcated the state into two Union Territories - Jammu and Kashmir and Ladakh.\r Pakistan termed the Indian action as "unilateral and illegal", and said it will take the matter to the UN Security Council.\r "The Secretary-General has been following the situation in Jammu and Kashmir with concern and makes an appeal for maximum restraint," Guterres\' spokesman Stephane Dujarric said here.\r Dujarric specifically said that the Secretary-General "also recalls the 1972 Agreement on bilateral relations between India and Pakistan, also known as the Simla Agreement, which states that the final status of Jammu and Kashmir is to be settled by peaceful means" in accordance with the Charter of the United Nations.\r The Secretary General did not offer his good offices nor did he make any offer to mediate between India and Pakistan on Kashmir. Instead, he referre

In [0]:
link2 = "https://timesofindia.indiatimes.com/india/article-370-did-nothing-but-encourage-separatism-terrorism-dynastic-rule-corruption-pm-modi/articleshow/70596459.cms"

In [0]:
import requests
res=requests.get(link2)
soup = BeautifulSoup(res.text,"html.parser")

In [23]:
soup.findAll('a')

[<a href="https://navbharattimes.indiatimes.com/" rel="nofollow noopener noreferrer" target="_blank">à¤¹à¤¿à¤¨à¥à¤¦à¥</a>,
 <a href="https://maharashtratimes.indiatimes.com" rel="nofollow noopener noreferrer" target="_blank">à¤®à¤°à¤¾à¤ à¥</a>,
 <a href="https://eisamay.indiatimes.com" rel="nofollow noopener noreferrer" target="_blank">à¦¬à¦¾à¦à¦²à¦¾<!-- --> </a>,
 <a href="https://www.iamgujarat.com/" rel="nofollow noopener noreferrer" target="_blank">àªà«àªàª°àª¾àª¤à«</a>,
 <a href="https://vijaykarnataka.indiatimes.com/" rel="nofollow noopener noreferrer" target="_blank">à²à²¨à³à²¨à²¡</a>,
 <a href="https://tamil.samayam.com" rel="nofollow noopener noreferrer" target="_blank">à®¤à®®à®¿à®´à¯</a>,
 <a href="https://telugu.samayam.com" rel="nofollow noopener noreferrer" target="_blank">à°¤à±à°²à±à°à±</a>,
 <a href="https://malayalam.samayam.com" rel="nofollow noopener noreferrer" target="_blank">à´®à´²à´¯à´¾à´³à´</a>,
 <a href="http://epaperbeta.timesofindia.com/" rel="

In [24]:
def parser(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,"html.parser")
  result = soup.find_all("a")
  text = [result[i].text for i in range(len(result))]
  final_news =" ".join(text)
  return final_news
te=parser(news)
print(te)


 #AajSawaroApnaKal SUBMIT    

 

     LIVE TV DownloadNews18 App English हिन्दी বাংলা मराठी ગુજરાતી ಕನ್ನಡ தமிழ் മലയാളം తెలుగు ਪੰਜਾਬੀ اردو অসমীয়া Home Elections Politics India Opinion Entertainment Tech Auto Buzz Videos Cricket TV Shows TV Videos Travel Business Health and Fitness Blogs Podcast Immersives Football Food Stocks Lok Sabha Elections 2019 Assembly Elections 2018 Services World Lifestyle Mission Paani Budget Sports iReel Awards Latest Ayodhya Verdict Photos Lifestyle Immersives Vanity Diaries ISL 2019 Lifestyle & Banking Swasth India Privacy Policy English English हिन्दी Hindi বাংলা Bengali मराठी Marathi ગુજરાતી Gujarati ಕನ್ನಡ Kannada தமிழ் Tamil മലയാളംMalayalam తెలుగుTELUGU ਪੰਜਾਬੀPunjabi اردو Urdu        Home Elections Politics India Opinion Entertainment Tech Auto Buzz Videos Cricket All Sections    LIVE TV     Home Elections Politics India Opinion Entertainment Tech Auto Buzz Videos Cricket TV Shows TV Videos Travel Business Health and Fitness Blogs Podcast Immersives Fo

## Latent Semantic Analysis / Topic Modelling


In [25]:
df=pd.read_csv("/content/drive/My Drive/NLP datasets/Consumer_Complaints_big.csv")
df = df[["Consumer complaint narrative"]]
df = df.dropna()
df_new = df[:100]
df_new = df_new["Consumer complaint narrative"].str.lower()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from string import punctuation
from nltk.corpus import stopwords
stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
stemmer=LancasterStemmer()
corpus =df_new.tolist()
dataset1=[]
final_corpus=[]
for x in range(len(corpus)):
    text=word_tokenize(corpus[x].lower())
    text = [stemmer.stem(y) for y in text if y not in stuff_to_be_removed]
    ent = " ".join(text)
    final_corpus.append(ent)
vectorizer = TfidfVectorizer()
X= vectorizer.fit_transform(final_corpus)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=4,n_iter=100)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=4, n_iter=100,
             random_state=None, tol=0.0)

In [0]:
row = lsa.components_[0]
prob=list(row)

In [0]:
fenames = list(vectorizer.get_feature_names())

In [29]:
for i,y in enumerate(lsa.components_):
  word=zip(fenames,y)
  sortedwords=sorted(word,key=lambda x:x[1],reverse=True)
  sortedwords=sortedwords[:10]
  print("\n Topic",i,":")
  for x in sortedwords:
    print(x)


 Topic 0 :
('xxxx', 0.812724861834828)
('xx', 0.23372680027068216)
('report', 0.153079554529978)
('account', 0.13991230481337485)
('credit', 0.13721854299038336)
('debt', 0.11410284848532509)
('00', 0.09908409827516594)
('consum', 0.09133886984868748)
('inform', 0.09111075076089122)
('collect', 0.08824086853082351)

 Topic 1 :
('consum', 0.5626290552777313)
('block', 0.3007841576623223)
('theft', 0.26820243273317596)
('id', 0.2638454723897126)
('inform', 0.26005964663895004)
('ag', 0.20920214534272505)
('sect', 0.20300756198244616)
('report', 0.1910296805836367)
('victim', 0.1678314874432057)
('shal', 0.13850128650884916)

 Topic 2 :
('account', 0.3643729062190856)
('credit', 0.2292472744437237)
('belong', 0.18459952232211213)
('docu', 0.13922609001478972)
('would', 0.13192043257427877)
('debt', 0.12858469467954428)
('lik', 0.12681902680985918)
('report', 0.12193794679847002)
('sign', 0.12108763407055903)
('investigation', 0.11932608747191636)

 Topic 3 :
('victim', 0.5170040264316145

##Gensim Library

In [30]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 9.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=a60bdd121935aefb3e551a3f9bc2dec38210f2a4cf67f8c4fd63a618d3ad94fb
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [33]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [34]:
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [35]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [36]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [37]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [38]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_poste', 'host', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [39]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1)]]


In [40]:
id2word[9]

'door'

In [41]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('specs', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [43]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


[(0,
  '0.036*"team" + 0.036*"game" + 0.020*"sale" + 0.019*"play" + 0.015*"hockey" '
  '+ 0.012*"year" + 0.008*"nhl" + 0.008*"trade" + 0.008*"wing" + '
  '0.007*"steven"'),
 (1,
  '0.040*"book" + 0.020*"belief" + 0.020*"atheist" + 0.018*"church" + '
  '0.015*"pin" + 0.015*"slave" + 0.014*"sphere" + 0.012*"character" + '
  '0.010*"lord" + 0.009*"headache"'),
 (2,
  '0.015*"choose" + 0.012*"input" + 0.011*"sin" + 0.010*"notice" + 0.009*"eat" '
  '+ 0.009*"cd" + 0.009*"food" + 0.009*"material" + 0.008*"signal" + '
  '0.008*"external"'),
 (3,
  '0.035*"not" + 0.022*"write" + 0.022*"do" + 0.020*"would" + 0.020*"line" + '
  '0.019*"organization" + 0.017*"be" + 0.017*"article" + 0.014*"get" + '
  '0.014*"know"'),
 (4,
  '0.533*"ax" + 0.008*"rlk" + 0.006*"cub" + 0.005*"echo" + '
  '0.004*"tufts_university" + 0.004*"stl" + 0.004*"pitcher" + 0.004*"pit" + '
  '0.004*"lk" + 0.003*"differential"'),
 (5,
  '0.027*"israel" + 0.015*"israeli" + 0.011*"jew" + 0.009*"lebanese" + '
  '0.009*"arab" + 0.00